In [1]:
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
import pickle
f = open("../data/ava/signif_genomic_feat_cond_json.pkl", 'rb')
signif_feat_cond_json = pickle.load(f)
signif_feat_cond_json[0:5]

[{'mutated features': 'C0343',
  'condition': 'LB',
  'odd ratio': 7.06415670600656,
  'p value': 2.3619591549177893e-49},
 {'mutated features': 'C0343',
  'condition': 'bla-TEM-1',
  'odd ratio': 8.182171226831422,
  'p value': 0.00047429908975170246},
 {'mutated features': 'C0343',
  'condition': 'ceftazidime',
  'odd ratio': 7.598065955864121,
  'p value': 6.086795351124303e-12},
 {'mutated features': 'C0343',
  'condition': 'pBGT',
  'odd ratio': 5.655200280900442,
  'p value': 8.157552538283797e-06},
 {'mutated features': 'C0343',
  'condition': 'ΔtpiA',
  'odd ratio': 8.73452661328309,
  'p value': 0.0003550714376402174}]

In [3]:
all_muts_df = pd.read_pickle("../data/ava/4_10_with_uniq_midpts.pkl")
all_muts_df.shape

(5676, 52)

In [4]:
def get_signif_mut_conds(feat_name):
    signif_mut_conds = []
    for signif_mut_cond in signif_feat_cond_json:
        if signif_mut_cond['mutated features'] == feat_name:
            signif_mut_conds.append(signif_mut_cond)
    return signif_mut_conds


temp_df = all_muts_df.copy()
temp_df["genomic features"] = temp_df.apply(lambda r: str(r["genomic features"]), axis=1)

cond_mut_feat_assoc_df = pd.DataFrame()
for _, mut in all_muts_df.iterrows():
    for feat in mut["genomic features"]:
        signif_mut_conds = get_signif_mut_conds(feat["name"])
        feat_name = str(feat["name"])
        df = temp_df[temp_df["genomic features"].str.contains(feat_name)]
        mut_cnt = len(df)
        exp_cnt = len(df["exp"].unique())
        for d in signif_mut_conds:
            d["mutation count"] = mut_cnt
            d["experiment count"] = exp_cnt
            cond_mut_feat_assoc_df = cond_mut_feat_assoc_df.append(d, ignore_index=True)

cond_mut_feat_assoc_df = cond_mut_feat_assoc_df.rename(columns={"mutated features": "mutated features"})
cond_mut_feat_assoc_df = cond_mut_feat_assoc_df.drop_duplicates()
display(len(cond_mut_feat_assoc_df), cond_mut_feat_assoc_df.head())

4277

,condition,experiment count,mutated features,mutation count,odd ratio,p value
0,42 celsius,6.0,nagA,10.0,8.203817,0.026243
1,glutaric acid,6.0,nagA,10.0,26.627148,0.000244
2,ΔtpiA glucose M9 pre-evolved,6.0,nagA,10.0,35.824798,0.001631
3,30 celsius,37.0,rph-pyrE attenuator terminator,132.0,2.017887,0.025572
4,42 celsius,37.0,rph-pyrE attenuator terminator,132.0,2.483604,0.018831


In [5]:
cond_cnt_d = {"none": 0}
for _, mut in all_muts_df.iterrows():
    for feat in mut["genomic features"]:
        if len(feat["significantly associated conditions"]) > 0:
            for cond in feat["significantly associated conditions"]:
                if cond in cond_cnt_d.keys():                   
                    cond_cnt_d[cond] += 1
                else:
                    cond_cnt_d[cond] = 1
        else:
            cond_cnt_d["none"] += 1

cond_cnt_d

{'none': 2322,
 '42 celsius': 198,
 'CaCl2(0.1)': 811,
 'MgSO4(0.24)': 805,
 'NaCl(0.5g/L) trace elements': 1624,
 'NH4Cl(1)': 466,
 'M9': 811,
 'KH2PO4(3) Na2HPO4(6.8)': 805,
 'glucose(4)': 659,
 'WT': 674,
 'U-13C glucose(2)': 46,
 'chloramphenicol(25)': 60,
 'ethylene glycol(30mM)': 60,
 'E. coli K-12 MG1655 pCA24N-fucO': 60,
 '30 celsius': 326,
 'glycerol(2.5)': 146,
 'glucose(2)': 1523,
 'FeSO4(20uM) sodium citrate(10mM)': 18,
 'isobutanol(8g/L)': 484,
 'glucose(3)': 587,
 'JCL260': 484,
 'LB': 1653,
 'D-2-deoxyribose': 60,
 'D-arabinose': 15,
 'D-lyxose': 22,
 'm-tartrate': 137,
 'monomethyl succinate': 13,
 'bla-TEM-1': 207,
 'ceftazidime': 980,
 'pBGT': 670,
 'chloramphenicol': 52,
 'doxycycline': 43,
 'trimethoprim': 23,
 'PQ(0.8mM)': 8,
 'glucose-evolved BOP1000': 8,
 'Δpgi': 89,
 "glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1)": 25,
 'ΔsdaA ΔsdaB ΔtdcG ΔglyA': 25,
 'acetate(4)': 25,
 'glucose(4) or acetate(4)': 78,
 'glucose(4) or glycerol(2.5)': 

In [6]:
signif_odds_assoc_mat = pd.DataFrame(0.0,
    index=cond_mut_feat_assoc_df["mutated features"].unique(),
    columns=cond_mut_feat_assoc_df["condition"].unique()
).astype(float)

for _, r in cond_mut_feat_assoc_df.iterrows():
    if r["p value"] <= 0.05:
        signif_odds_assoc_mat.at[r["mutated features"], r["condition"]] = r["odd ratio"]
signif_odds_assoc_mat

,42 celsius,glutaric acid,ΔtpiA glucose M9 pre-evolved,30 celsius,CaCl2(0.1),FeSO4(20uM) sodium citrate(10mM),GMOS,H. sapien pgi,H. sapien tpiA,KH2PO4(3) Na2HPO4(6.8),M9,MgSO4(0.24),NH4Cl(1),NaCl(0.5g/L) trace elements,U-13C glucose(2),V. cholerae pgi,V. cholerae tpiA,WT,adipic acid,chloramphenicol,glucose(4),glucose(4) or acetate(4),glucose(4) or glycerol(2.5),n-butanol,xylose(4),(NH4)2SO4(1),(NH4)2SO4(1) MgSO4,K2HPO4(5.3) KH2PO4(2),glucose(1),thiaminne hydrochloride(0.3µM) Ga(NO3)3(100mg/L),2 3-butanediol,B. melitensis pgi,hexanoic acid,octanoic acid,putrescine,CaCl2(0.1mM),KH2PO4(3) Na2HPO4(6.78),M63,MS57,MgSO4(1mM),glucose(2),hexamethylenediamine,Δpgi,m-tartrate,coumaric acid,JCL260,LB,glucose(3),isobutanol(8g/L),(NH4)2SO4(2),(NH4)2SO4(2) MgSO4(0.12),C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins,C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins,KH2PO4(13.6),none,P. aerophilum pgi,isobutyric acid,gentamycin(0.03),doxycycline,C13H10N2O(0.168184),D-arabinose,acetate(4),glycerol(2.5),bla-TEM-1,ceftazidime,pBGT,ΔtpiA,Δgnd glucose M9 pre-evolved,glucose(4) or xylose(4),PQ(0.8mM),glucose-evolved BOP1000,D-lyxose,monomethyl succinate,E. coli K-12 MG1655 pCA24N-fucO,chloramphenicol(25),ethylene glycol(30mM),trimethoprim,ΔptsHI-crr glucose M9 pre-evolved,propanediol,ΔsdhCDAB glucose M9 pre-evolved,Δpgi glucose M9 pre-evolved,glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1),ΔsdaA ΔsdaB ΔtdcG ΔglyA,P. aerophilum tpiA,D-2-deoxyribose
nagA,8.203817,26.627148,35.824798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph-pyrE attenuator terminator,2.483604,0.000000,0.000000,2.017887,1.568975,13.949127,9.292362,5.810807,5.693974,1.550407,1.568975,1.550407,1.263700,1.233108,13.331591,7.335756,15.874179,1.508563,4.292962,7.273606,1.770056,7.084826,15.183832,5.380277,5.938240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph,4.810688,0.000000,0.000000,3.546273,1.576537,26.924309,17.923450,0.000000,8.668622,1.533742,1.576537,1.533742,0.000000,1.328012,8.589891,0.000000,0.000000,1.550213,0.000000,11.001487,2.367957,0.000000,10.339222,0.000000,11.469884,9.269346,9.269346,9.269346,9.269346,9.269346,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rpoC,0.000000,5.087940,0.000000,0.000000,1.536935,0.000000,0.000000,0.000000,3.339890,1.545373,1.536935,1.545373,1.283897,0.000000,0.000000,11.325113,0.000000,1.570300,0.000000,0.000000,0.000000,6.578805,0.000000,0.000000,6.730018,0.000000,0.000000,0.000000,0.000000,0.000000,5.604316,5.569066,6.212055,9.790708,3.890582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hfq,13.867592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.521855,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.082324,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [7]:
# remove columns and rows of just 0s

signif_odds_assoc_mat = signif_odds_assoc_mat.loc[:, (signif_odds_assoc_mat != 0).any(axis=0)]  # remove all columns with only 0s
signif_odds_assoc_mat = signif_odds_assoc_mat.loc[(signif_odds_assoc_mat != 0).any(axis=1)]
signif_odds_assoc_mat

,42 celsius,glutaric acid,ΔtpiA glucose M9 pre-evolved,30 celsius,CaCl2(0.1),FeSO4(20uM) sodium citrate(10mM),GMOS,H. sapien pgi,H. sapien tpiA,KH2PO4(3) Na2HPO4(6.8),M9,MgSO4(0.24),NH4Cl(1),NaCl(0.5g/L) trace elements,U-13C glucose(2),V. cholerae pgi,V. cholerae tpiA,WT,adipic acid,chloramphenicol,glucose(4),glucose(4) or acetate(4),glucose(4) or glycerol(2.5),n-butanol,xylose(4),(NH4)2SO4(1),(NH4)2SO4(1) MgSO4,K2HPO4(5.3) KH2PO4(2),glucose(1),thiaminne hydrochloride(0.3µM) Ga(NO3)3(100mg/L),2 3-butanediol,B. melitensis pgi,hexanoic acid,octanoic acid,putrescine,CaCl2(0.1mM),KH2PO4(3) Na2HPO4(6.78),M63,MS57,MgSO4(1mM),glucose(2),hexamethylenediamine,Δpgi,m-tartrate,coumaric acid,JCL260,LB,glucose(3),isobutanol(8g/L),(NH4)2SO4(2),(NH4)2SO4(2) MgSO4(0.12),C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins,C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins,KH2PO4(13.6),none,P. aerophilum pgi,isobutyric acid,gentamycin(0.03),doxycycline,C13H10N2O(0.168184),D-arabinose,acetate(4),glycerol(2.5),bla-TEM-1,ceftazidime,pBGT,ΔtpiA,Δgnd glucose M9 pre-evolved,glucose(4) or xylose(4),PQ(0.8mM),glucose-evolved BOP1000,D-lyxose,monomethyl succinate,E. coli K-12 MG1655 pCA24N-fucO,chloramphenicol(25),ethylene glycol(30mM),trimethoprim,ΔptsHI-crr glucose M9 pre-evolved,propanediol,ΔsdhCDAB glucose M9 pre-evolved,Δpgi glucose M9 pre-evolved,glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1),ΔsdaA ΔsdaB ΔtdcG ΔglyA,P. aerophilum tpiA,D-2-deoxyribose
nagA,8.203817,26.627148,35.824798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph-pyrE attenuator terminator,2.483604,0.000000,0.000000,2.017887,1.568975,13.949127,9.292362,5.810807,5.693974,1.550407,1.568975,1.550407,1.263700,1.233108,13.331591,7.335756,15.874179,1.508563,4.292962,7.273606,1.770056,7.084826,15.183832,5.380277,5.938240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph,4.810688,0.000000,0.000000,3.546273,1.576537,26.924309,17.923450,0.000000,8.668622,1.533742,1.576537,1.533742,0.000000,1.328012,8.589891,0.000000,0.000000,1.550213,0.000000,11.001487,2.367957,0.000000,10.339222,0.000000,11.469884,9.269346,9.269346,9.269346,9.269346,9.269346,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rpoC,0.000000,5.087940,0.000000,0.000000,1.536935,0.000000,0.000000,0.000000,3.339890,1.545373,1.536935,1.545373,1.283897,0.000000,0.000000,11.325113,0.000000,1.570300,0.000000,0.000000,0.000000,6.578805,0.000000,0.000000,6.730018,0.000000,0.000000,0.000000,0.000000,0.000000,5.604316,5.569066,6.212055,9.790708,3.890582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hfq,13.867592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.521855,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.082324,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [8]:
import numpy as np
signif_logodds_assoc_mat = np.log(signif_odds_assoc_mat)
signif_logodds_assoc_mat = signif_logodds_assoc_mat.replace(-np.inf, 0)
signif_logodds_assoc_mat

/home/pphaneuf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


,42 celsius,glutaric acid,ΔtpiA glucose M9 pre-evolved,30 celsius,CaCl2(0.1),FeSO4(20uM) sodium citrate(10mM),GMOS,H. sapien pgi,H. sapien tpiA,KH2PO4(3) Na2HPO4(6.8),M9,MgSO4(0.24),NH4Cl(1),NaCl(0.5g/L) trace elements,U-13C glucose(2),V. cholerae pgi,V. cholerae tpiA,WT,adipic acid,chloramphenicol,glucose(4),glucose(4) or acetate(4),glucose(4) or glycerol(2.5),n-butanol,xylose(4),(NH4)2SO4(1),(NH4)2SO4(1) MgSO4,K2HPO4(5.3) KH2PO4(2),glucose(1),thiaminne hydrochloride(0.3µM) Ga(NO3)3(100mg/L),2 3-butanediol,B. melitensis pgi,hexanoic acid,octanoic acid,putrescine,CaCl2(0.1mM),KH2PO4(3) Na2HPO4(6.78),M63,MS57,MgSO4(1mM),glucose(2),hexamethylenediamine,Δpgi,m-tartrate,coumaric acid,JCL260,LB,glucose(3),isobutanol(8g/L),(NH4)2SO4(2),(NH4)2SO4(2) MgSO4(0.12),C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins,C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins,KH2PO4(13.6),none,P. aerophilum pgi,isobutyric acid,gentamycin(0.03),doxycycline,C13H10N2O(0.168184),D-arabinose,acetate(4),glycerol(2.5),bla-TEM-1,ceftazidime,pBGT,ΔtpiA,Δgnd glucose M9 pre-evolved,glucose(4) or xylose(4),PQ(0.8mM),glucose-evolved BOP1000,D-lyxose,monomethyl succinate,E. coli K-12 MG1655 pCA24N-fucO,chloramphenicol(25),ethylene glycol(30mM),trimethoprim,ΔptsHI-crr glucose M9 pre-evolved,propanediol,ΔsdhCDAB glucose M9 pre-evolved,Δpgi glucose M9 pre-evolved,glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1),ΔsdaA ΔsdaB ΔtdcG ΔglyA,P. aerophilum tpiA,D-2-deoxyribose
nagA,2.104599,3.281931,3.57864,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph-pyrE attenuator terminator,0.909711,0.000000,0.00000,0.702051,0.450423,2.635417,2.229193,1.75972,1.739408,0.438517,0.450423,0.438517,0.234044,0.209537,2.590136,1.992760,2.764694,0.411158,1.456977,1.984252,0.571011,1.957955,2.720231,1.68274,1.781413,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph,1.570840,0.000000,0.00000,1.265897,0.455231,3.293030,2.886110,0.00000,2.159710,0.427710,0.455231,0.427710,0.000000,0.283683,2.150586,0.000000,0.000000,0.438392,0.000000,2.398030,0.862028,0.000000,2.335945,0.00000,2.439725,2.226713,2.226713,2.226713,2.226713,2.226713,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rpoC,0.000000,1.626873,0.00000,0.000000,0.429790,0.000000,0.000000,0.00000,1.205938,0.435266,0.429790,0.435266,0.249900,0.000000,0.000000,2.427023,0.000000,0.451266,0.000000,0.000000,0.000000,1.883853,0.000000,0.00000,1.906578,0.000000,0.000000,0.000000,0.000000,0.000000,1.723537,1.717227,1.826492,2.281434,1.358559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hfq,2.629555,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.72622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.808490,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [9]:
signif_logodds_assoc_mat.to_pickle("../data/assocs/geno_signif_logodds_assoc_mat.pkl")